***
## Полный список мороженого на главной странице

Обработкой запросов к главной странице занимается view-функция index() приложения homepage. Вот в ней-то и сделаем запрос к БД: «получить все поля всех объектов модели IceCream».

На SQL этот запрос выглядел бы так:

```sql
SELECT *
FROM <имя таблицы>; 
```

Но у нас есть Django ORM, так что запрос делаем не на SQL, а на Python:

```py
# homepage/views.py
from django.shortcuts import render

from ice_cream.models import IceCream

def index(request):
    template_name = 'homepage/index.html'
    # Запрос:
    ice_cream_list = IceCream.objects.all()
    # Полученный из БД QuerySet передаём в словарь контекста:
    context = {
        'ice_cream_list': ice_cream_list,
    }
    # Словарь контекста передаём в шаблон, рендерим HTML-страницу:
    return render(request, template_name, context)
```

Запрос будет отправлен, если в шаблоне будут запрошены данные из словаря `context`; добавим в HTML-шаблон переменную из этого словаря:

```html
 <!-- templates/homepage/index.html -->
...

<h1 class="pb-2 mb-0">Главная страница</h1>
{{ ice_cream_list }}

... 
```

И теперь, когда информация из БД востребована в шаблоне, при запросе к главной странице будет отправлен SQL-запрос:

![alt text](https://pictures.s3.yandex.net/resources/S3.2_36_1679044940.png)


Прямо в шаблоне можно проитерироваться по QuerySet, получить свойства каждого объекта и вывести их на страницу, например, в виде HTML-списка: 

```html
<!-- templates/homepage/index.html -->
...

{% for ice_cream in ice_cream_list %}
  <ul>
    <li>ID: {{ ice_cream.id }}</li>
    <li>Опубликовано: {{ ice_cream.is_published }}</li>
    <li>На главную: {{ ice_cream.is_on_main }}</li>
    <li>Название: {{ ice_cream.title }}</li>
    <li>Описание: {{ ice_cream.description }}</li>
    <li>FK wrapper: {{ ice_cream.wrapper_id }}</li>
    <li>FK category: {{ ice_cream.category_id }}</li>
  </ul>
{% endfor %} 
```

***
## Метод values(): берём только нужное

В QuerySet содержатся все поля объектов модели `IceCream`, но на странице, которая отправляется пользователю, совершенно не нужны поля `is_published`, `is_on_main`, `wrapper_id` и `category_id`: это техническая информация, не предназначенная для пользователя. Получается, мы напрасно запрашивали эти поля и передавали их в шаблон, создавая лишнюю нагрузку на сервер. 

Ограничим количество столбцов, попадающих в выборку. В SQL следовало бы сделать так:

```sql
SELECT <столбец1>, <столбец2>
FROM <имя таблицы>; 
```

А в Django ORM поля, которые нужно получить из БД, перечисляются в аргументах метода .values(): 

```py
<Название модели>.objects.values('<свойство1>', '<свойство2>') 
```

Обновите код view-функции:

```py
# homepage/views.py
from django.shortcuts import render

from ice_cream.models import IceCream

def index(request):
    template_name = 'homepage/index.html'
    # Возьмём нужное. А ненужное не возьмём:
    ice_cream_list = IceCream.objects.values('id', 'title')
    context = {
        'ice_cream_list': ice_cream_list,
    }
    return render(request, template_name, context)
```

При вызове метода `.values()` в QuerySet попадает не список объектов, а список словарей; ключами этого словаря служат названия свойств, а значениями — значения этих свойств. 

Измените шаблон, чтобы наблюдать за результатами не только в Django Debug Toolbar, но и на странице проекта:

```html
<!-- templates/homepage/index.html -->
...

{% for ice_cream in ice_cream_list %}
  <ul>
    <li>ID: {{ ice_cream.id }}</li>
    <li>Название: {{ ice_cream.title }}</li>
  </ul>
{% endfor %} 
```

Передавать в запросе только те данные, которые используются на странице, — правильный подход: чем меньше передаётся информации, тем быстрее всё работает.